In [7]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 05: Census

## Table Review: Welcome Survey

In [8]:
survey = Table.read_table('data/classdatasurvey_w24.csv')
survey
survey.where('What position do you sleep in?', are.contained('Back'))

Timestamp,What year are you in?,How would you rate yourself on a scale of 10 from 1 (introvert) to 10 (extrovert),How many texts do you send per day?,"Are you left-handed, right-handed or both?",What position do you sleep in?,How many hours of sleep do you get?,How many siblings do you have?,"What pets do you have? If more than one, separate with commas.",Make up a very random 4-digit number,Do you have a tattoo?,How do you get to work or school?
1/16/2024 15:14:27,First,4,20,Right,Stomach,7-8,1,0,2376,No,walk
1/16/2024 15:14:40,Second,3,5,Right,Back,6,1,cat,4635,No,walk
1/16/2024 15:15:23,Third,4,"15-30, depends",Right,Back,5-7,3,nan,3790,No,Bike
1/16/2024 15:17:50,Third,5,0,Right,Back,6h,no,no,6117,No,Bus
1/16/2024 15:17:51,First,2,"Depends on the day, between 5-30.",Right,Stomach,"On average, 5.",3,0,6782,Yes,walk
1/16/2024 15:19:07,First,2,5-30,Right,Stomach,5,3,0,6782,Yes,walk
1/16/2024 15:22:39,First,5,nan,Right,Left side,6-7 :(,2 :D,I have two cats,5939,Yes,Bike
1/16/2024 15:23:32,First,2,70,Right,Right side,5,2,nan,3276,No,Bus
1/16/2024 15:25:20,Third,6,20,Right,Right side,6-7,1,1,2323,Yes,Bike
1/16/2024 15:25:39,First,5,nan,Right,Left side,6-7 :(,Two,Two cats! :D,3216,Yes,Bike


In [18]:
Table.drop?

Signature: Table.drop(self, *column_or_columns)
Docstring:
Return a Table with only columns other than selected label or
labels.

Args:
    ``column_or_columns`` (string or list of strings): The header
    names or indices of the columns to be dropped.

    ``column_or_columns`` must be an existing header name, or a
    valid column index.

Returns:
    An instance of ``Table`` with given columns removed.

>>> t = Table().with_columns(
...     'burgers',  make_array('cheeseburger', 'hamburger', 'veggie burger'),
...     'prices',   make_array(6, 5, 5),
...     'calories', make_array(743, 651, 582))
>>> t
burgers       | prices | calories
cheeseburger  | 6      | 743
hamburger     | 5      | 651
veggie burger | 5      | 582
>>> t.drop('prices')
burgers       | calories
cheeseburger  | 743
hamburger     | 651
veggie burger | 582
>>> t.drop(['burgers', 'calories'])
prices
6
5
5
>>> t.drop('burgers', 'calories')
prices
6
5
5
>>> t.drop([0, 2])
prices
6
5
5
>>> t.drop(0, 2)
prices
6
5
5
>>>

In [25]:
import pandas
survey_w24 = pandas.read_csv('data/classdatasurvey_w24.csv')
survey_w24 = survey_w24.rename( columns = { "What year are you in?": "Year",
                                        "How would you rate yourself on a scale of 10 from 1 (introvert) to 10 (extrovert)":"Extraversion",
                                        "How many texts do you send per day?": "Number of texts",
                                        "Are you left-handed, right-handed or both?":"Handedness",
                                        "What position do you sleep in?" : "Sleep position",
                                        "How many hours of sleep do you get?" : "Hours of Sleep",
                                        "How many siblings do you have?" : "Siblings",
                                        "What pets do you have? If more than one, separate with commas.": "Pets",
                                        "Make up a very random 4-digit number" : "Random Number" ,
                                        "Do you have a tattoo?" : "Tattoo",
                                        "How do you get to work or school?" : "Commute"})
survey_w24 = survey_w24.drop('Timestamp', axis = 1)

survey_w24['Number of texts'] = pandas.to_numeric(survey_w24['Number of texts'], errors = 'coerce', downcast = 'integer')
survey_w24['Hours of Sleep'] = pandas.to_numeric(survey_w24['Hours of Sleep'], errors = 'coerce', downcast = 'integer')
survey_w24 = survey_w24.dropna() # dropped rows with NaN values
type(survey_w24) # pandas dataframe
survey_w24 = Table().from_df(survey_w24) # converting data frame to datascience table
type(survey_w24) # datascience Table
survey_w24.to_csv('data/welcome_survey_w24_cleaned.csv') # wrote new table to a CSV file, stored it in the data folder
                                        

In [29]:
survey = Table.read_table('data/welcome_survey_w24_cleaned.csv') # reading in the new csv file, into a datascience Table
survey_w24 = survey.select('Year', 'Extraversion', 'Hours of Sleep', 'Handedness', 'Sleep position', 'Pets')
len(survey_w24.column('Hours of Sleep')) # had to delete a bunch of NaN values, so now we have 51 unique entries in survey_w24


51

### Find average hours of sleep for side sleepers (hint: use [where-predicates](http://www.data8.org/datascience/reference-nb/datascience-reference.html#Table.where-Predicates))

In [30]:
np.mean(survey_w24.column('Hours of Sleep')) # average hours of sleep for all sleep positions

7.5490196078431371

In [32]:
# look for side sleepers
side_sleepers = survey_w24.where('Sleep position', are.containing('side'))
side_sleepers
# then find average
np.mean(side_sleepers.column('Hours of Sleep'))


7.2972972972972974

### Average hours of sleep for back sleepers

In [33]:
# look for back sleepers
back_sleepers = survey_w24.where('Sleep position', are.containing('Back'))
back_sleepers
# then find average
np.mean(back_sleepers.column('Hours of Sleep'))


7.625

### Share of students sleeping at least 8 hours

In [38]:
sleeps_a_lot = survey_w24.where('Hours of Sleep', are.above_or_equal_to(8))
# to calculate proportion, i want to find number of students who sleep a lot, divided by total number of students in my table
number_students_sleep_a_lot = sleeps_a_lot.num_rows
total_number_students = survey_w24.num_rows
total_number_students
proportion_students_sleep_a_lot = number_students_sleep_a_lot / total_number_students
proportion_students_sleep_a_lot

0.45098039215686275

### Subset table with only sleep information

In [41]:
Sleep_data = survey_w24.select('Hours of Sleep', 'Sleep position') # displays the columns you selected
# if you want to create a new table with just these two columns, we need to assign its value to a new table name

Sleep_data

Hours of Sleep,Sleep position
6,Back
8,Left side
8,Back
6,Left side
8,Back
8,Stomach
7,Right side
10,Right side
8,Right side
9,Back


## Discussion Question: NBA Salaries

The table nba has columns PLAYER, POSITION, and SALARY

**Tasks:**
- Rename "'15-'16 SALARY" to "SALARY" using `relabeled()`
- Subset the table to only contain the player name, the position and the salary

In [ ]:
nba = Table.read_table('data/nba_salaries.csv')

**Tasks:**
- Create an array containing the **names** of all point guards (PG) who make more than 15 MUSD/year. Salaries are given in MUSD

**Question**:

What does the following return

```python
nba.drop('POSITION')
nba.num_columns
```

---
Back to slides

---

## Census: Population Trends

[A description of the table appears online.](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/nc-est2019-agesex-res.pdf)

The Census dataset contain estimates of the US population in each category of sex and age in the years 2020 through 2022. The Census is decennial: it takes place every 10 years. The most recent Census was held in 2020 and the one before that in 2010. The Census Bureau also estimates population changes each year. As explained in the Bureau’s description of its methodology, it “adds the estimated changes to the last decennial census to produce updated population estimates every year.”

In [42]:
data_location = 'http://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/nc-est2019-agesex-res.csv'
data_location = 'https://www2.census.gov/programs-surveys/popest/datasets/2020-2022/national/asrh/nc-est2022-agesex-res.csv'
data_location = 'data/nc-est2019-agesex-res.csv'
data_location = 'data/nc-est2022-agesex-res.csv'

census = Table.read_table(data_location)
census.sort('AGE')

SEX,AGE,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022
0,0,3690644,3669863,3570131,3683113
1,0,1886273,1876349,1825237,1882867
2,0,1804371,1793514,1744894,1800246
0,1,3731823,3719267,3675382,3588817
1,1,1906537,1899515,1879051,1834594
2,1,1825286,1819752,1796331,1754223
0,2,3826588,3807454,3725597,3693362
1,2,1957418,1946596,1902665,1888149
2,2,1869170,1860858,1822932,1805213
0,3,3918009,3885476,3813656,3742836


### Subset the table
for now, we are only interested in the population size by sex (`SEX`), age (`AGE`) in 2020 and 2022 (`POPESTIMATE2020`, and `POPESTIMATE2022|`).

**Tasks**:
- `POPESTIMATE2020` and `POPESTIMATE2022` are cumbersome. Lets rename them to `2020` and `2022`
- Create a new table containing only the columns we are interested in

In [45]:
census_2020_2022 = census.select('SEX','AGE','POPESTIMATE2020', 'POPESTIMATE2022')
census_2020_2022 = census_2020_2022.relabeled('POPESTIMATE2020', '2020').relabeled( 'POPESTIMATE2022', '2022')
# the line up above, relabeling one column at a time
# we relabel twice
census_2020_2022

SEX,AGE,2020,2022
0,0,3669863,3683113
0,1,3719267,3588817
0,2,3807454,3693362
0,3,3885476,3742836
0,4,3987970,3830225
0,5,4023811,3907298
0,6,4012049,4008898
0,7,3999001,4043729
0,8,4006214,4031341
0,9,4050595,4017929


**Tasks:**
- How many people above the age of 100 were there in the US in 2020 and 2022?
- How about an age of above 99

**Question:**
- Why don't we see the number of people at age of 101+?


In [47]:
census_2020_2022.sort('AGE', descending = True)

SEX,AGE,2020,2022
0,999,331511512,333287557
1,999,164525109,165283553
2,999,166986403,168004004
0,100,81905,88988
1,100,14861,17085
2,100,67044,71903
0,99,51403,53870
1,99,11230,12345
2,99,40173,41525
0,98,73442,78028


---
Back to slides

---

### Sex ratio
The Census form asks respondents to provide the sex of each household member by checking one of two boxes labeled Male and Female. The SEX column contains numeric codes: 1 for male, 2 for female, and 0 for the total.

**Task**:Find ratio of females to males in the US population in 2022
- Remember that `999` contains all ages

In [ ]:
# try on your own
# ration 50/50 = 1 /1
# 50:50 or 1:1

# proportion 50/100

**Task**
- Now lets check the ratio for different age groups

**Task:**
- Create a new table with columns for age and ratios
    - Create a table with only females
    - Create a table with only males
    - Get the age column from either
- plot

### Age change
**Tasks**
- let's load the 2019 census data
- Lets compare the change of population per age group.
- Calculate the absolute change
- Calcluate the relative change

In [ ]:
data_location = 'data/nc-est2019-agesex-res.csv'

census = Table.read_table(data_location)
census.sort('AGE')

**Tasks**
- Create a new table with the ages and the age changes
- Don't include the *all age row**
- Plot the population change vs the age

Take a look at the top few rows. While the percent change is about 3% for the overall population, it jumps to well over 20% for the people in their late sixties and early seventies. This stunning change contributes to what is known as the greying of America.

What could explain this large increase? 

<details><summary>Answer</summary>
We can explore this question by examining the years in which the relevant groups were born.

Those who were in the age group 69 to 72 in 2014 were born in the years 1942 to 1945. The attack on Pearl Harbor was in late 1941, and by 1942 U.S. forces were heavily engaged in a massive war that ended in 1945.

Those who were 69 to 72 years old in 2019 were born in the years 1947 to 1950, at the height of the post-WWII baby boom in the United States.

The post-war jump in births is a major reason for the large changes that we have observed.
</details>